## Initialize

In [1]:

import requests
import pandas as pd
import os
import sys
from datetime import datetime

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path
import src.esh_client as esh

base_url = get_base_url()
root_path = get_root_path()
example_name = sys.path[0].split(os.sep)[-2]


def execute_search_query(query):
    r = requests.post(
        f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[
                ['title', 'text', 'createdAt', 'changedAt']]
            df.rename(columns={'title': 'Title',
                      'text': 'Text','createdAt':'Created at', 'changedAt':'Changed at' }, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())


## Text search

In [21]:
so = esh.EshObject(
    count=True,
    top=3,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='BMW')
            )
        ]
    )
)
execute_search_query(so.dict(exclude_none=True))

Error during search. HTTP status code 404
{'detail': "Tennant id 'DocResearchDemoExample' does not exist"}


## Search for several text snippets

In [13]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='BMW Finance')
            ),
            
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Finance'))
        ]
    )
)
execute_search_query(so.dict(exclude_none=True))

Total count: 3


,Title,Text
0,mercedes_report,JVBERi0xLjYNJeLjz9MNCjQ5NjMgMCBvYmoNPDwvRmlsdG...
1,bmw_report,JVBERi0xLjcNJeLjz9MNCjE1ODMgMCBvYmoNPDwvTGluZW...
2,volkswagen_report,JVBERi0xLjcNJeLjz9MNCjUyNjMyIDAgb2JqDTw8L0xpbm...


## Search for explicit numbers

In [19]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='2,340', isPhrase=True)
            )]
    )
)
execute_search_query(so.dict(exclude_none=True))

Total count: 2


,Title,Text
0,audi_report,JVBERi0xLjcNCiW1tbW1DQoxIDAgb2JqDQo8PC9UeXBlL0...
1,audi_report,JVBERi0xLjcNCiW1tbW1DQoxIDAgb2JqDQo8PC9UeXBlL0...
